In [ ]:
import sys; sys.path.append('..')

In [ ]:
import math
import pickle
from pathlib import Path
from tqdm import tqdm
from util import max_size2possible_t, ds_names

In [ ]:
p_incident = Path('../data/incident')
p_core = Path('../data/core')
p_core.mkdir(exist_ok=True)
for ds_name in ds_names:
    p_incident_ds = p_incident / ds_name
    p_core_ds = p_core / ds_name
    p_core_ds.mkdir(exist_ok=True)
    with (p_incident_ds / 'i2edges.pkl').open('rb') as f:
        i2edges = pickle.load(f)
    with (p_incident_ds / 'v2edges.pkl').open('rb') as f:
        v2edges = pickle.load(f)
    n, m = len(v2edges), len(i2edges)
    max_size = max(len(e) for e in i2edges.values())
    possible_t = max_size2possible_t[max_size]
    # v2t_k = defaultdict(list)  # v2t_k[v] = [[t, C_t(v)] for t in possible_t]
    print('Dataset {ds}, # possible t = {nt}'.format(ds=ds_name, nt=len(possible_t)))
    for i, t in tqdm(list(enumerate(possible_t))):
        v2coreness_t = dict()
        if i:
            with (p_incident_ds / 'i2edges.pkl').open('rb') as f:
                i2edges = pickle.load(f)
            with (p_incident_ds / 'v2edges.pkl').open('rb') as f:
                v2edges = pickle.load(f)
        i2th = dict()
        for i_e, e in i2edges.items():
            i2th[i_e] = max(math.ceil(t * len(e)), 2)
        k = None
        nodes_to_remove = set()
        while i2edges:
            if not nodes_to_remove:
                k = min(len(E_v) for E_v in v2edges.values()) + 1
                nodes_to_remove = set()
                for v, E_v in v2edges.items():
                    if len(E_v) == k - 1:
                        nodes_to_remove.add(v)
                        v2coreness_t[v] = k - 1
                continue
            edges_to_check = set()
            for v in nodes_to_remove:
                edges_to_check.update(v2edges[v])
                del v2edges[v]
            assert edges_to_check.issubset(i2edges)
            nodes_to_remove_next = set()
            for i_e in edges_to_check:
                i2edges[i_e] -= nodes_to_remove
                if len(i2edges[i_e]) < i2th[i_e]:
                    for v in i2edges[i_e]:
                        if v not in v2edges:
                            continue
                        if len(v2edges[v]) == k:
                            nodes_to_remove_next.add(v)
                            v2coreness_t[v] = k - 1
                        v2edges[v].remove(i_e)
                    del i2edges[i_e]
            nodes_to_remove = nodes_to_remove_next.copy()
        assert len(v2coreness_t) == n
        with (p_core_ds / 'c_{}.pkl'.format(str(t).replace('/', '-'))).open('wb') as f:
            pickle.dump(v2coreness_t, f)
        # for v, k in v2coreness_t.items():
        #     v2t_k[v].append([t, k])
    # with (p_core_ds / 'v2tk.pkl').open('wb') as f:
    #     pickle.dump(v2t_k, f)